In [1]:
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from datasets import Dataset, DatasetDict
# from peft import (
#     LoraConfig,
#     prepare_model_for_kbit_training,
#     get_peft_model
# )
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
import json

In [2]:
# 假设 module.py 中有一些函数和变量
# 你想要导入所有的函数

# 导入整个 module.py
import eval_fewshot

# 获取 module 中所有的函数
from inspect import getmembers
from types import FunctionType

# 创建一个字典，包含 module 中所有的函数
all_functions = {name: obj for name, obj in getmembers(eval_fewshot) if isinstance(obj, FunctionType)}

In [3]:
import pprint
from eval_fewshot import *
p = pprint.pprint
p(getmembers(eval_fewshot))
p(all_functions)

[('AutoModelForCausalLM',
  <class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>),
 ('AutoTokenizer',
  <class 'transformers.models.auto.tokenization_auto.AutoTokenizer'>),
 ('CodeGenTokenizer', <class 'tokenization_codegen.CodeGenTokenizer'>),
 ('Dict', typing.Dict),
 ('IGNORE_INDEX', -100),
 ('MistralForCausalLM',
  <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>),
 ('PhiForCausalLM', <class 'modeling_phi.PhiForCausalLM'>),
 ('SentenceTransformer',
  <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>),
 ('Sequence', typing.Sequence),
 ('Tuple', typing.Tuple),
 ('__builtins__',
  {'ArithmeticError': <class 'ArithmeticError'>,
   'AssertionError': <class 'AssertionError'>,
   'AttributeError': <class 'AttributeError'>,
   'BaseException': <class 'BaseException'>,
   'BlockingIOError': <class 'BlockingIOError'>,
   'BrokenPipeError': <class 'BrokenPipeError'>,
   'BufferError': <class 'BufferError'>,
   'BytesWarning': <cla

In [10]:
parser = argparse.ArgumentParser()

parser.add_argument('--data_path', type=str, default="/opt/tiger/HKU-DASC7606-A2/data/ARC-Easy-test.jsonl")
parser.add_argument('--device_id', type=str, default="0,1,2,3,4,5,6,7")
parser.add_argument('--model', type=str, default='allenai/unifiedqa-t5-large', help="")
parser.add_argument('--embedder', type=str, default="BAAI/bge-small-en-v1.5")
parser.add_argument('--output_path', type=str, help="")
parser.add_argument('--start_index', type=int, default=0, help="")
parser.add_argument('--end_index', type=int, default=164, help="")
parser.add_argument('--N', type=int, default=8, help="")
parser.add_argument('--max_len', type=int, default=512, help="")
parser.add_argument('--overwrite', type=str2bool, default=False, help="")
parser.add_argument('--prompt_type', type=str, default="v1.0", help="")
parser.add_argument('--top_k', type=str2bool, default=False, help="")
parser.add_argument('--top_k_reverse', type=str2bool, default=False, help="")

args = """--model 'allenai/unifiedqa-t5-large' --embedder "BAAI/bge-small-en-v1.5" --data_path "data/ARC-Challenge-test.jsonl" --start_index 0 --end_index 9999 --max_len 1024 --output_path "test_mistral" --overwrite False --prompt_type "v2.0" --N 8 --top_k True --top_k_reverse False""".replace("\"","").replace("\'","").split(" ")
args = parser.parse_args(args)
p(args)

Namespace(data_path='data/ARC-Challenge-test.jsonl', device_id='0,1,2,3,4,5,6,7', model='allenai/unifiedqa-t5-large', embedder='BAAI/bge-small-en-v1.5', output_path='test_mistral', start_index=0, end_index=9999, N=8, max_len=1024, overwrite=False, prompt_type='v2.0', top_k=True, top_k_reverse=False)


In [11]:
data_path = "/opt/tiger/HKU/data/ARC-Easy-test.jsonl"
problems = all_functions["get_arc_problems"](data_path)[0: 9999]

In [12]:
p(len(problems))
p(problems[3])

9501
{'answer': 'The producers in all ecosystems are plants.',
 'answerKey': 'A',
 'candidate_answers': '(A) Sunlight is the source of energy for nearly all '
                      'ecosystems. (B) Most ecosystems are found on land '
                      'instead of in water. (C) Carbon dioxide is more '
                      'available than other gases. (D) The producers in all '
                      'ecosystems are plants.',
 'id': 'Mercury_417466',
 'label': 'D',
 'question': 'Which statement best explains why photosynthesis is the '
             'foundation of most food webs?'}


In [13]:
num_samples = len(problems)
tokenizer, model = get_mistral(base_model=args.model)
embedder = SentenceTransformer(args.embedder)
demonstrations = load_all_demonstrations(data_path.replace("test", "train")) #作为prompt的一部分, (question, choices text, choices label, answerkey)
demonstration_embeddings = llm_embedder(embedder, [d[0] for d in demonstrations], False) # ndarray: [n_demons, n_dim], 将 questions 转为 vector 进行检索

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, RwkvConfig, Speech2Text2Config, TransfoXLConfig, TrO

In [9]:
p(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

In [ ]:
p(demonstrations[0])

('Which factor will most likely cause a person to develop a fever?',
 ['a leg muscle relaxing after exercise',
  'a bacterial population in the bloodstream',
  'several viral particles on the skin',
  'carbohydrates being digested in the stomach'],
 ['A', 'B', 'C', 'D'],
 'B')


In [ ]:
question = problems[2]["question"]
answer = problems[2]["answer"]
candidate_answers = problems[2]["candidate_answers"]

source = generate_prompt(question, candidate_answers, args.prompt_type, args.N,
                            demonstrations, demonstration_embeddings, embedder,
                            top_k=args.top_k, top_k_reverse=args.top_k_reverse)

target = " {}".format(answer)
encoding = mistral_preprocess([source], [target], tokenizer, args)

with torch.no_grad():
    outputs = model(**encoding)
    log_likelihood = outputs.loss * -1

ValueError: only one element tensors can be converted to Python scalars

In [57]:
p(source)
p(target)
p(question)

('Question: Which observation of a plant supports the inference that '
 'photosynthesis can take place?\n'
 'Answer: a green color\n'
 '\n'
 'Question: Where do plants get energy from to make food?\n'
 'Answer: sunlight\n'
 '\n'
 'Question: Green plants get the energy they need to make food from\n'
 'Answer: sunlight\n'
 '\n'
 'Question: What happens during photosynthesis?\n'
 'Answer: Plants change light energy into food energy.\n'
 '\n'
 'Question: Which is the best explanation of the term ecology?\n'
 'Answer: the study of organisms and their interactions with the environment\n'
 '\n'
 'Question: Which structure is found in producers in nearly all ecosystems?\n'
 'Answer: chloroplasts\n'
 '\n'
 'Question: What supplies most of the energy that plants need to make food?\n'
 'Answer: sunlight\n'
 '\n'
 'Question: Which of the following occurs during photosynthesis?\n'
 'Answer: CO2 and H2O are converted to carbohydrates.\n'
 '\n'
 'Question: Which statement best explains why photosynth

In [58]:
p(outputs)
p(log_likelihood)

{'logits': tensor([[[13.9531, 14.5547, 10.7656,  ...,  2.0020,  2.0039,  2.0039],
         [11.1875, 10.6016, 10.4375,  ...,  2.8125,  2.8145,  2.8145],
         [ 8.3203,  7.0273,  4.6367,  ...,  2.5527,  2.5527,  2.5547],
         ...,
         [ 4.7266,  6.4492,  2.6406,  ...,  0.3752,  0.3765,  0.3757],
         [18.0781, 13.6484, 11.8516,  ...,  4.7656,  4.7656,  4.7617],
         [20.2188, 14.1094, 18.7188,  ..., 11.4062, 11.4062, 11.4062]]],
       device='cuda:0'),
 'loss': tensor(3.7866, device='cuda:0'),
 'past_key_values': ((tensor([[[[ 3.1763e-01,  3.3569e-01, -2.3672e+00,  ..., -1.5420e+00,
           -4.9878e-01, -6.5771e-01],
          [-7.3242e-01, -8.3008e-02, -6.9482e-01,  ..., -6.4160e-01,
            2.7612e-01, -3.1799e-02],
          [ 2.4453e+00,  1.1035e+00,  1.4219e+00,  ...,  8.6377e-01,
            1.7197e+00,  6.1230e-01],
          ...,
          [-2.9121e+00,  3.1172e+00,  2.0762e+00,  ..., -1.7646e+00,
            2.5527e+00,  3.4302e-01],
          [-1.2